### Install libraries

In [ ]:
import csv
!pip3 install tensorflow==1.15.4
!pip3 install pandas

In [ ]:
# get bert layer
!pip3 install bert-experimental --no-deps

In [ ]:
!pip3 install tensorflow_hub==0.7.0

In [ ]:
!pip3 install 'h5py==2.8.0' --force-reinstall
!pip3 install 'numpy==1.18.5' --force-reinstall

### Evaluation

#### Twitter

In [ ]:
import pandas as pd
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph

df = pd.read_csv('test_data_two_columns.csv', sep=":")
total_posts = df[['post']].count().post
res = [2] * total_posts
for i in range(0, total_posts):
    found_gender = 0
    chunk = []
    post = ''.join(df[['post']].values[i])
    exceptions = ("http", "Источник", "художественный перевод", "Википедии:",".ru", "ЖЖ")
    if not(any(word in post for word in exceptions)):
        for sent in sentenize(post):
            tokens = [_.text for _ in tokenize(sent.text)]
            chunk.append(tokens)
        navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
        morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
        morph.navec(navec)
        cnt_chunk = chunk.copy()

        for sent in chunk:
            ya_in_quotes = 0
            if sent.count('"') > 1:
                cnt_el_in_sent = 0
                total_quotes = sent.count('"')
                while total_quotes > 1: # Change яЯ in quotes
                    open_quote = sent.index('"', cnt_el_in_sent)
                    close_quote = sent.index('"', open_quote + 1)
                    while 'я' in sent and open_quote < sent.index('я') < close_quote or \
                            'Я' in sent and open_quote < sent.index('Я') < close_quote:
                        if 'я' in sent and open_quote < sent.index('я') < close_quote:
                            sent[sent.index('я')] = 'ya'
                            ya_in_quotes += 1
                        elif 'Я' in sent and open_quote < sent.index('Я') < close_quote:
                            sent[sent.index('Я')] = 'YA'
                            ya_in_quotes += 1
                    total_quotes -= 2
                    cnt_el_in_sent = close_quote + 1
        m = 0   # for gender 1
        w = 0   # for gender 0
        for sent in chunk:
            if not(sent.count('-') > 0 and sent.index('-') < 3):
                if 'я' in sent or 'Я' in sent:
                    if 'я' in sent:
                        ya_index = sent.index('я')
                    else:
                        ya_index = sent.index('Я')
                    markup = next(morph.map(cnt_chunk))
                    for v in sent:
                        if (v[-2:] == 'ла' or v[-1:] == 'л' or v[-3:] == 'лся' or v[-4:] == 'лась') \
                                and (markup.tokens[sent.index(v)].pos == 'VERB') \
                                and ya_index <= sent.index(v) <= ya_index + 5 \
                                and (markup.tokens[sent.index(v)].feats.get('Gender') is not None):
                            found_gender = 1
                            if  v[-2:] == 'ла' or v[-4:] == 'лась':
                                w += 1
                            elif v[-1:] == 'л' or v[-3:] == 'лся' :
                                m += 1
                            break
            cnt_chunk.pop(0)
        if found_gender == 1:
            if m > w:
                res[i] = 1
            elif w > m:
                res[i] = 0

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from bert_experimental.finetuning.bert_layer import BertLayer


df1 = pd.read_csv("test_data_two_columns.csv",sep=":")
label_encoder1 = LabelEncoder()
df1['label'] = label_encoder1.fit_transform(df1.gender.tolist())

X1 = df1.post.values
L1 = df1.label.values

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model("/Users/dbadeev/PycharmProjects/gender_cls_copy/cls_model_256.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
new_model.summary()

In [ ]:
tsp1 = new_model.predict(X1, verbose=1)

In [ ]:
import numpy as np

predicted_classes1 = np.argmax(tsp1,axis=1)
predicted_classes1_upd = np.argmax(tsp1,axis=1)

# check for stylometric diff with model prediction
for i in range (0, len(predicted_classes1)):
    if res[i] != predicted_classes1[i] and res[i] != 2:
        print("DIFF - ", i, res[i], " stylo - model ", predicted_classes1[i], "\n")

for i in range (0, len(predicted_classes1)):
    if res[i] != predicted_classes1_upd[i] and res[i] != 2:
        predicted_classes1_upd[i] = res[i]

In [ ]:
# check for stylometric diff with true value
for i in range (0, len(L1)):
    if res[i] != L1[i] and res[i] != 2:
        print("ALARM!!! - ", i, res[i], L1[i], "\n")

In [ ]:
#model prediction
print(classification_report(L1, predicted_classes1))

In [ ]:
#model+stylometry prediction
print(classification_report(L1, predicted_classes1_upd))

#### Facebook

In [ ]:
import pandas as pd
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph

df = pd.read_csv('fb_data_two_columns.csv', sep=":")
total_posts = df[['post']].count().post
res = [2] * total_posts
for i in range(0, total_posts):
    found_gender = 0
    chunk = []
    post = ''.join(df[['post']].values[i])
    exceptions = ("http", "Источник", "художественный перевод", "Википедии:",".ru", "ЖЖ")
    if not(any(word in post for word in exceptions)):
        for sent in sentenize(post):
            tokens = [_.text for _ in tokenize(sent.text)]
            chunk.append(tokens)
        navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
        morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
        morph.navec(navec)
        cnt_chunk = chunk.copy()

        for sent in chunk:
            ya_in_quotes = 0
            if sent.count('"') > 1:
                cnt_el_in_sent = 0
                total_quotes = sent.count('"')
                while total_quotes > 1: # Change яЯ in quotes
                    open_quote = sent.index('"', cnt_el_in_sent)
                    close_quote = sent.index('"', open_quote + 1)
                    while 'я' in sent and open_quote < sent.index('я') < close_quote or \
                            'Я' in sent and open_quote < sent.index('Я') < close_quote:
                        if 'я' in sent and open_quote < sent.index('я') < close_quote:
                            sent[sent.index('я')] = 'ya'
                            ya_in_quotes += 1
                        elif 'Я' in sent and open_quote < sent.index('Я') < close_quote:
                            sent[sent.index('Я')] = 'YA'
                            ya_in_quotes += 1
                    total_quotes -= 2
                    cnt_el_in_sent = close_quote + 1
        m = 0   # for gender 1
        w = 0   # for gender 0
        for sent in chunk:
            if not(sent.count('-') > 0 and sent.index('-') < 3):
                if 'я' in sent or 'Я' in sent:
                    if 'я' in sent:
                        ya_index = sent.index('я')
                    else:
                        ya_index = sent.index('Я')
                    markup = next(morph.map(cnt_chunk))
                    for v in sent:
                        if (v[-2:] == 'ла' or v[-1:] == 'л' or v[-3:] == 'лся' or v[-4:] == 'лась') \
                                and (markup.tokens[sent.index(v)].pos == 'VERB') \
                                and ya_index <= sent.index(v) <= ya_index + 5 \
                                and (markup.tokens[sent.index(v)].feats.get('Gender') is not None):
                            found_gender = 1
                            if  v[-2:] == 'ла' or v[-4:] == 'лась':
                                w += 1
                            elif v[-1:] == 'л' or v[-3:] == 'лся' :
                                m += 1
                            break
            cnt_chunk.pop(0)
        if found_gender == 1:
            if m > w:
                res[i] = 1
            elif w > m:
                res[i] = 0

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from bert_experimental.finetuning.bert_layer import BertLayer

df1 = pd.read_csv("fb_data_two_columns.csv",sep=":")
label_encoder1 = LabelEncoder()
df1['label'] = label_encoder1.fit_transform(df1.gender.tolist())

X1 = df1.post.values
L1 = df1.label.values

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model("/Users/dbadeev/PycharmProjects/gender_cls_copy/cls_model_256.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model("/Users/dbadeev/PycharmProjects/gender_cls_copy/cls_middle_model_256.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
tsp1 = new_model.predict(X1, verbose=1)

In [ ]:
import numpy as np

predicted_classes1 = np.argmax(tsp1,axis=1)
predicted_classes1_upd = np.argmax(tsp1,axis=1)

# check for stylometric diff with model prediction
for i in range (0, len(predicted_classes1)):
    if res[i] != predicted_classes1[i] and res[i] != 2:
        print("DIFF - ", i, res[i], " stylo - model ", predicted_classes1[i], "\n")

for i in range (0, len(predicted_classes1)):
    if res[i] != predicted_classes1_upd[i] and res[i] != 2:
        predicted_classes1_upd[i] = res[i]

In [ ]:
# check for stylometric diff with true value
for i in range (0, len(L1)):
    if res[i] != L1[i] and res[i] != 2:
        print("ALARM!!! - ", i, res[i], L1[i], "\n")

In [ ]:
#model prediction
print(classification_report(L1, predicted_classes1))

In [ ]:
#model+stylometry prediction
print(classification_report(L1, predicted_classes1_upd))

#### Essays

In [ ]:
import pandas as pd
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph

df = pd.read_csv('essays_data_two_columns.csv', sep=":")
total_posts = df[['post']].count().post
res = [2] * total_posts
for i in range(0, total_posts):
    found_gender = 0
    chunk = []
    post = ''.join(df[['post']].values[i])
    exceptions = ("http", "Источник", "художественный перевод", "Википедии:",".ru", "ЖЖ")
    if not(any(word in post for word in exceptions)):
        for sent in sentenize(post):
            tokens = [_.text for _ in tokenize(sent.text)]
            chunk.append(tokens)
        navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
        morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
        morph.navec(navec)
        cnt_chunk = chunk.copy()

        for sent in chunk:
            ya_in_quotes = 0
            if sent.count('"') > 1: # Change яЯ in quotes
                cnt_el_in_sent = 0
                total_quotes = sent.count('"')
                while total_quotes > 1:
                    open_quote = sent.index('"', cnt_el_in_sent)
                    close_quote = sent.index('"', open_quote + 1)
                    while 'я' in sent and open_quote < sent.index('я') < close_quote or \
                            'Я' in sent and open_quote < sent.index('Я') < close_quote:
                        if 'я' in sent and open_quote < sent.index('я') < close_quote:
                            sent[sent.index('я')] = 'ya'
                            ya_in_quotes += 1
                        elif 'Я' in sent and open_quote < sent.index('Я') < close_quote:
                            sent[sent.index('Я')] = 'YA'
                            ya_in_quotes += 1
                    total_quotes -= 2
                    cnt_el_in_sent = close_quote + 1
        m = 0   # for gender 1
        w = 0   # for gender 0
        for sent in chunk:
            if not(sent.count('-') > 0 and sent.index('-') < 3):
                if 'я' in sent or 'Я' in sent:
                    if 'я' in sent:
                        ya_index = sent.index('я')
                    else:
                        ya_index = sent.index('Я')
                    markup = next(morph.map(cnt_chunk))
                    for v in sent:
                        if (v[-2:] == 'ла' or v[-1:] == 'л' or v[-3:] == 'лся' or v[-4:] == 'лась') \
                                and (markup.tokens[sent.index(v)].pos == 'VERB') \
                                and ya_index <= sent.index(v) <= ya_index + 5 \
                                and (markup.tokens[sent.index(v)].feats.get('Gender') is not None):
                            found_gender = 1
                            if  v[-2:] == 'ла' or v[-4:] == 'лась':
                                w += 1
                            elif v[-1:] == 'л' or v[-3:] == 'лся' :
                                m += 1
                            break
            cnt_chunk.pop(0)
        if found_gender == 1:
            if m > w:
                res[i] = 1
            elif w > m:
                res[i] = 0

In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from bert_experimental.finetuning.bert_layer import BertLayer

df1 = pd.read_csv("essays_data_two_columns.csv",sep=":")
label_encoder1 = LabelEncoder()
df1['label'] = label_encoder1.fit_transform(df1.gender.tolist())

X1 = df1.post.values
L1 = df1.label.values

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model("/Users/dbadeev/PycharmProjects/gender_cls_copy/cls_model_256.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model("/Users/dbadeev/PycharmProjects/gender_cls_copy/cls_big_model_256.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
tsp1 = new_model.predict(X1, verbose=1)

In [ ]:
import numpy as np

predicted_classes1 = np.argmax(tsp1,axis=1)
predicted_classes1_upd = np.argmax(tsp1,axis=1)

# check for stylometric diff with model prediction
for i in range (0, len(predicted_classes1)):
    if res[i] != predicted_classes1[i] and res[i] != 2:
        print("DIFF - ", i, res[i], " stylo - model ", predicted_classes1[i], "\n")

for i in range (0, len(predicted_classes1)):
    if res[i] != predicted_classes1_upd[i] and res[i] != 2:
        predicted_classes1_upd[i] = res[i]

In [ ]:
# check for stylometric diff with true value
for i in range (0, len(L1)):
    if res[i] != L1[i] and res[i] != 2:
        print("ALARM!!! - ", i, res[i], L1[i], "\n")

In [ ]:
#model prediction
print(classification_report(L1, predicted_classes1))

In [ ]:
#model+stylometry prediction
print(classification_report(L1, predicted_classes1_upd))

#### Reviews

In [ ]:
import pandas as pd
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph

df = pd.read_csv('reviews_data_two_columns.csv', sep=":")
total_posts = df[['post']].count().post
res = [2] * total_posts
for i in range(0, total_posts):
    found_gender = 0
    chunk = []
    post = ''.join(df[['post']].values[i])
    exceptions = ("http", "Источник", "художественный перевод", "Википедии:",".ru", "ЖЖ")
    if not(any(word in post for word in exceptions)):
        for sent in sentenize(post):
            tokens = [_.text for _ in tokenize(sent.text)]
            chunk.append(tokens)
        navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
        morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
        morph.navec(navec)
        cnt_chunk = chunk.copy()

        for sent in chunk:
            ya_in_quotes = 0
            if sent.count('"') > 1: # Change яЯ in quotes
                cnt_el_in_sent = 0
                total_quotes = sent.count('"')
                while total_quotes > 1:
                    open_quote = sent.index('"', cnt_el_in_sent)
                    close_quote = sent.index('"', open_quote + 1)
                    while 'я' in sent and open_quote < sent.index('я') < close_quote or \
                            'Я' in sent and open_quote < sent.index('Я') < close_quote:
                        if 'я' in sent and open_quote < sent.index('я') < close_quote:
                            sent[sent.index('я')] = 'ya'
                            ya_in_quotes += 1
                        elif 'Я' in sent and open_quote < sent.index('Я') < close_quote:
                            sent[sent.index('Я')] = 'YA'
                            ya_in_quotes += 1
                    total_quotes -= 2
                    cnt_el_in_sent = close_quote + 1
        m = 0   # for gender 1
        w = 0   # for gender 0
        for sent in chunk:
            if not(sent.count('-') > 0 and sent.index('-') < 3):
                if 'я' in sent or 'Я' in sent:
                    if 'я' in sent:
                        ya_index = sent.index('я')
                    else:
                        ya_index = sent.index('Я')
                    markup = next(morph.map(cnt_chunk))
                    for v in sent:
                        if (v[-2:] == 'ла' or v[-1:] == 'л' or v[-3:] == 'лся' or v[-4:] == 'лась') \
                                and (markup.tokens[sent.index(v)].pos == 'VERB') \
                                and ya_index <= sent.index(v) <= ya_index + 5 \
                                and (markup.tokens[sent.index(v)].feats.get('Gender') is not None):
                            found_gender = 1
                            if  v[-2:] == 'ла' or v[-4:] == 'лась':
                                w += 1
                            elif v[-1:] == 'л' or v[-3:] == 'лся' :
                                m += 1
                            break
            cnt_chunk.pop(0)
        if found_gender == 1:
            if m > w:
                res[i] = 1
            elif w > m:
                res[i] = 0

In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from bert_experimental.finetuning.bert_layer import BertLayer


df1 = pd.read_csv("reviews_data_two_columns.csv",sep=":")
label_encoder1 = LabelEncoder()
df1['label'] = label_encoder1.fit_transform(df1.gender.tolist())

X1 = df1.post.values
L1 = df1.label.values

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model("/Users/dbadeev/PycharmProjects/gender_cls_copy/cls_model_256.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model("/Users/dbadeev/PycharmProjects/gender_cls_copy/cls_big_model_256.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
tsp1 = new_model.predict(X1, verbose=1)

In [ ]:
import numpy as np

predicted_classes1 = np.argmax(tsp1,axis=1)
predicted_classes1_upd = np.argmax(tsp1,axis=1)

# check for stylometric diff with model prediction
for i in range (0, len(predicted_classes1)):
    if res[i] != predicted_classes1[i] and res[i] != 2:
        print("DIFF - ", i, res[i], " stylo - model ", predicted_classes1[i], "\n")


for i in range (0, len(predicted_classes1)):
    if res[i] != predicted_classes1_upd[i] and res[i] != 2:
        predicted_classes1_upd[i] = res[i]

In [ ]:
# check for stylometric diff with true value
for i in range (0, len(L1)):
    if res[i] != L1[i] and res[i] != 2:
        print("ALARM!!! - ", i, res[i], L1[i], "\n")

In [ ]:
#model prediction
print(classification_report(L1, predicted_classes1))

In [ ]:
#model+stylometry prediction
print(classification_report(L1, predicted_classes1_upd))

#### Imitations

In [ ]:
df1 = pd.read_csv("imitation_full_data_two_columns.csv",sep=":")
label_encoder1 = LabelEncoder()
df1['label'] = label_encoder1.fit_transform(df1.gender.tolist())

X1 = df1.post.values
L1 = df1.label.values

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model("cls_model.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
from tensorflow.keras.models import load_model

suoer_big_model = load_model("cls_super_big_model.h5", custom_objects={'BertLayer': BertLayer})

In [ ]:
tsp1 = new_model.predict(X1, verbose=1)

In [ ]:
predicted_classes1 = np.argmax(tsp1,axis=1)

In [ ]:
#model prediction
print(classification_report(L1, predicted_classes1))
